# Analyses statistiques de la base Agribalyse

## Import des packages

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

import pandas as pd
import numpy as np

from tqdm import tqdm # package pour afficher des barres de progression

## Chargement des données

In [2]:
# Utilisation du module créé dans "loader.py"

import sys
from pathlib import Path

# Ajout du chemin vers les modules
project_root = Path.cwd().parent
sys.path.append(str(project_root / "src"))

# Import de la fonction
from ecomenu_assistant.data.loader import load_agribalyse_data

# Chargement des données
data = load_agribalyse_data()

# Vérification du chargement
print(f"Données chargées: {len(data)} lignes, {len(data.columns)} colonnes")
data.head()

Chargement des données depuis: /Users/davidbugalho/Desktop/ecomenu-assistant/data/raw/Agribalyse_Synthese.csv
Données chargées: 14956 lignes, 32 colonnes
Après nettoyage: 2451 lignes utiles
Colonnes sélectionnées: 7
Données chargées: 2451 lignes, 7 colonnes


,Nom du Produit en Français,Groupe d'aliment,Sous-groupe d'aliment,Changement climatique,DQR,code saison,code avion
0,"Court-bouillon pour poissons, déshydraté",aides culinaires et ingrédients divers,aides culinaires,7.58,2.24,2.0,0.0
1,"Pizza, sauce garniture pour",aides culinaires et ingrédients divers,aides culinaires,0.94,2.19,2.0,0.0
2,"Préparation culinaire à base de soja, type ""cr...",aides culinaires et ingrédients divers,aides culinaires,1.17,1.75,2.0,0.0
3,"Agar (algue), cru",aides culinaires et ingrédients divers,algues,11.80,2.77,2.0,0.0
4,"Ao-nori (Enteromorpha sp.), séchée ou déshydratée",aides culinaires et ingrédients divers,algues,11.80,2.77,2.0,0.0


## Analyse

### Informations générales

In [3]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 2451 entries, 0 to 2450
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Nom du Produit en Français  2451 non-null   string 
 1   Groupe d'aliment            2451 non-null   string 
 2   Sous-groupe d'aliment       2451 non-null   string 
 3   Changement climatique       2451 non-null   float64
 4   DQR                         2451 non-null   float64
 5   code saison                 2451 non-null   float64
 6   code avion                  2451 non-null   float64
dtypes: float64(4), string(3)
memory usage: 153.2 KB


,Changement climatique,DQR,code saison,code avion
count,2451.000000,2451.000000,2451.000000,2451.000000
mean,5.764182,2.461334,1.997552,0.000816
std,8.115851,0.673588,0.063841,0.028560
min,0.000342,1.000000,0.000000,0.000000
25%,1.370000,1.960000,2.000000,0.000000
50%,2.780000,2.550000,2.000000,0.000000
75%,6.570000,2.940000,2.000000,0.000000
max,52.500000,4.790000,2.000000,1.000000


### Analyses des groupes d'aliments

In [4]:
data.groupby("Groupe d'aliment")["Changement climatique"].agg(['count','mean','min','max']).sort_values('mean',ascending=False)

,count,mean,min,max
Groupe d'aliment,,,,
"viandes, œufs, poissons",648,13.119182,0.540000,52.50
produits sucrés,53,7.063038,0.746000,19.30
entrées et plats composés,309,4.800861,0.100000,27.80
lait et produits laitiers,223,4.461883,1.150000,12.50
matières grasses,58,4.342414,1.630000,27.40
aides culinaires et ingrédients divers,157,3.857408,0.372000,13.20
produits céréaliers,375,2.661952,0.310000,10.30
glaces et sorbets,25,2.094400,1.000000,4.24
aliments infantiles,38,1.749921,0.683000,8.05


In [5]:
data.groupby(["Groupe d'aliment", "Sous-groupe d'aliment"])["Changement climatique"].agg(['count','mean','min','max']).sort_values('mean',ascending=False).reset_index()

,Groupe d'aliment,Sous-groupe d'aliment,count,mean,min,max
0,"viandes, œufs, poissons",viandes cuites,105,25.678286,2.140000,52.400
1,"viandes, œufs, poissons",mollusques et crustacés cuits,7,18.808571,5.920000,41.600
2,"viandes, œufs, poissons",viandes crues,135,16.754370,1.500000,41.300
3,"viandes, œufs, poissons",autres produits à base de viande,14,15.207857,3.870000,52.500
4,"viandes, œufs, poissons",mollusques et crustacés crus,14,12.301429,5.330000,39.200
...,...,...,...,...,...,...
56,boissons,boissons sans alcool,95,1.159716,0.043600,29.100
57,produits sucrés,"sucres, miels et assimilés",7,1.156000,0.746000,1.920
58,"fruits, légumes, légumineuses et oléagineux",légumineuses,36,0.740806,0.409000,2.200
59,aides culinaires et ingrédients divers,sels,5,0.632000,0.632000,0.632


### Produits extrêmes

In [6]:
print("Top 10 des produits les plus polluants:")
data.nlargest(10, 'Changement climatique')[['Nom du Produit en Français', 'Changement climatique']]

Top 10 des produits les plus polluants:


,Nom du Produit en Français,Changement climatique
1804,Brochette d'agneau,52.5
2347,"Agneau, côte filet, grillée/poêlée",52.4
2348,"Agneau, côte première, grillée/poêlée",52.4
2349,"Agneau, côtelette, grillée",52.4
2353,"Agneau, gigot, grillé/poêlé",52.4
2355,"Agneau, selle, partie maigre, grillée/poêlée",52.4
2391,"Chevreau, cuit",52.4
2346,"Agneau, collier, braisé ou bouilli",52.3
2352,"Agneau, gigot, braisé",52.3
2354,"Agneau, gigot, rôti/cuit au four",52.3


In [7]:
print("Top 10 des produits les moins polluants:")
data.nsmallest(10, 'Changement climatique')[['Nom du Produit en Français', 'Changement climatique']]

Top 10 des produits les moins polluants:


,Nom du Produit en Français,Changement climatique
332,Eau du robinet,0.000342
324,"Thé infusé, non sucré",0.043600
325,"Thé noir, infusé, non sucré",0.043600
326,"Thé oolong, infusé, non sucré",0.043600
327,"Thé vert, infusé, non sucré",0.043600
328,"Tisane infusée, non sucrée",0.043600
601,"Bouillon de légumes, déshydraté reconstitué",0.100000
602,"Bouillon de volaille, déshydraté reconstitué",0.103000
600,"Bouillon de boeuf, déshydraté reconstitué",0.113000
271,Boisson préparée à partir de sirop à diluer ty...,0.125000
